In [1]:
import os
import sys
import exp
import pandas as pd

# requirements for paralell computing
'''
from multiprocessing import Process, freeze_support
from threading import Thread
from time import sleep
'''

# Processing folder path
src_path = sys.path[0].replace("exp_notebook", "src") # path for source code
data_path = sys.path[0].replace("exp_notebook", "data") # path for raw data
if src_path not in sys.path:
    sys.path.append(src_path)
input_path = sys.path[0].replace("exp_notebook", "exp_input") # path for the experiment log
log_folder = sys.path[0].replace("exp_notebook", "exp_log") # path for the experiment log

In [2]:

exp_suffix="demo" # identifier of the current experiment
exp_catagory='experiments' # path of parameter files
params_file="demo_params.xlsx" # file name of the parameter file

params_folder=os.path.join(input_path,exp_catagory)
assert os.path.exists(params_folder)

params_path = os.path.join(params_folder, params_file) # file path for the experiment parameters

# folder for saving the experiment logs
save_path = os.path.join(log_folder,exp_suffix) 
assert os.path.exists(save_path)

params_pd=pd.read_excel(params_path)

In [3]:
params_pd.head(5)

,id,status,bld,pv,ev,method,strategy,B_kWh,ev_to_bld,pred_model,...,bat_e_terminal,bat_e_terminal_revenue,load_bld_error_max_neg,load_bld_error_max_pos,load_pv_error_max_neg,load_pv_error_max_pos,load_ev_error_max_neg,load_ev_error_max_pos,load_net_error_max_neg,load_net_error_max_pos
0,1,0,Sum,Sum,ALL,MPC,optimal,438,0,GT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,Sum,Sum,ALL,RBC,MSC,438,0,GT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0,Sum,Sum,ALL,MPC,optimal,438,0,Prediction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,Sum,Sum,ALL,MPC,optimal,438,0,Prediction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0,Sum,Sum,ALL,MPC,optimal,438,0,Prediction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Parameters of the simulations are loaded from the file designated by `params_file`, which is an excel file under the folder `exp_input\experiments`.

Each row in the file represents one trial.
The explanations for these parameters are as follows:
- `status`: The status of the simulation trial;
- `bld`: Which building in the data set is included. _Sum_ means representative aggregate virtual profile;
- `pv`: Simlilar to `bld`;
- `ev`: Source data of which parking lot is used. When set as _ALL_, all available data is included.
- `method`: The optimization method used, including _MPC_ and _RBC_;
- `strategy`: The sub-method used. (i.e., for _RBC_, _MSC_ and _TOU_ can be chosen.);
- `B_kWh`: Battery size (kWh);
- `ev_to_bld`: Ratio of average power consumption of sampled EV sessions to average building load;
- `pred_model`: Method used for getting load forecasts. _GT_ means ground truth, _Prediction_ refers to machine learning methods, _Simple_ refers to the heuristic method (Adaptive Moving Average (AMA) in this study), _Disturbance_ means the artificial noise. All methods are introduced in the manusript;
- `deg_model_opt` and `deg_model`: Parameters for the batteries (Not implemented in this study);
- `start` and `end`: The start date and end date of the simulation. Formatted as 'mm-dd' in the year of 2019;
- `p_grid_max`: The maximum power exchange with the external grid - The float set here times the average building load;
- `price_dc`: The tariff rate of ***Demand charge*** - The float set here times the days of the simulation period;
- `price_sell`: The price of selling power to the external grid - The float set here times the current TOU tariff;
- `ev_charge_rule`: How are EVs charged for EV sessions unseen for the optimizer;
- `disturbance_rule`: The rule for generating artificial noise as introduced in the manuscript. _uniform_ refers to **U**, _uniform_pos_ refers to **U+** and _uniform_neg_ refers to **U-**;
- `disturbance_scale`: The magnitude of the artificial noise;

In [4]:
class MPC_exp(exp.MPC_ExperimentManager):
    None

em = MPC_exp(log_fn=params_path, save_path=save_path, save=True, exp_prefix="Demo")

# Designate the parameters used in this simulation
var_keys = ["method", "strategy",
            "pred_model", "deg_model_opt", "p_grid_max", 
            "price_dc", "price_sell", 
            "ev_charge_rule", 
            "B_kWh", "deg_model", 
            "start", "end", 
            "bld", "ev", "pv",
            "disturbance_rule", "disturbance_scale",
            "pred_method"
            ]
# num_trials: How many trials will be run
# fork_id: Identifier of subprocess when implementing multiprocessing
em.run(keys=var_keys, num_trials=3, fork_id=None)

!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
self.op_params[K]: 96
==================== 2019-05-01 00:00:00 thread_id: None ====================
summary saved at: 2019-05-01 00:00:00
==================== 2019-05-01 00:00:00 thread_id: None ====================
==================== 2019-05-02 00:00:00 thread_id: None ====================
==================== 2019-05-03 00:00:00 thread_id: None ====================
==================== 2019-05-04 00:00:00 thread_id: None ====================
==================== 2019-05-05 00:00:00 thread_id: None ====================
==================== 2019-05-06 00:00:00 thread_id: None ====================
==================== FINISH ====================
Done, trial 1
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
self.op_params[K]: 96
==================== 2019-05-01 00:00:00 thread_id: None ====================
summary saved at: 2019-05-01 00:00:00
==================== 2019-05-01 00:00:00 thread_id: None ==================

After running the simulations, corresponding logs can be found under `save_path` as designated aforehead.

In addition, the key summary of each trial is logged back to the params file. The summary includes:
- _OPEX_: Operational expenditure;
- _demand_charge_: Demand charge;
- _tou_cost_: TOU cost;
- _load_bld_: Total building load in the trial;
- _load_pv_ and _load_ev_: Similar to _load_bld_;
- _save_fn_: File name of the detailed log under `save_path`;